Математика в контексте задачи классификации

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import preprocessing
import category_encoders as ce

from sklearn import model_selection

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble

from sklearn import metrics

БЭГГИНГ

In [3]:
data = pd.read_csv('data/weatherAUS.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [4]:
data.isna().sum().sum()

343248

In [5]:
cols_isna_percent = data.isna().mean() * 100
cols_with_null = cols_isna_percent[cols_isna_percent>40].sort_values(ascending=False)
display(cols_with_null.index)
data = data.drop(cols_with_null.index, axis=1)

Index(['Sunshine', 'Evaporation', 'Cloud3pm'], dtype='object')

In [6]:
data['RainToday'] = data['RainToday'].apply(lambda x: 1 if x == 'Yes' else x)
data['RainToday'] = data['RainToday'].apply(lambda x: 0 if x == 'No' else x)
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else x)
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 0 if x == 'No' else x)

In [7]:
data['RainToday'].mean()

0.22419285648984874

In [8]:
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month

data = data.drop(['Date'], axis=1)

grouped = data.groupby(['Month']).mean()
grouped['RainToday'].sort_values(ascending=False)

Month
7     0.270736
6     0.263638
8     0.253167
9     0.229135
5     0.222163
3     0.217135
4     0.216845
12    0.213037
11    0.210843
2     0.206746
10    0.196512
1     0.189484
Name: RainToday, dtype: float64

In [9]:
data.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustDir',
       'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am',
       'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am',
       'Pressure3pm', 'Cloud9am', 'Temp9am', 'Temp3pm', 'RainToday',
       'RainTomorrow', 'Month'],
      dtype='object')

In [10]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data_dummies = pd.get_dummies(data, columns=categoricals)
data_dummies.shape

(145460, 124)

In [11]:
data_dummies.columns[data_dummies.columns.str.contains('Month')]

Index(['Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12'],
      dtype='object')

In [12]:
droped_data = data_dummies.dropna(how='any', axis=0)

In [13]:
X = droped_data.drop(['RainTomorrow'], axis=1)
y = droped_data['RainTomorrow']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=31)

In [14]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data))) #определяем индексы случайным образом
    numbers = data[inds] #выбираем значения по индексам
    return numbers
target = X_train['MinTemp'].values #выбираем целевую переменную
np.random.seed(31) #задаём параметр генератора случайных чисел
mean_values = [np.mean(x) for x in gbs(target, 1000)] #получаем все средние значения
np.std(mean_values) #находим для них стандартное отклонение

0.02879072820657669

In [15]:
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train, y_train)

y_train_pred_logreg = log_reg.predict(X_train)
y_test_pred_logreg = log_reg.predict(X_test)

print(metrics.roc_auc_score(y_test, y_test_pred_logreg))

0.7274030225534013


c:\Users\Пользователь\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
dt = tree.DecisionTreeClassifier(random_state=42)

params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

grid_search = model_selection.GridSearchCV(
    estimator=dt,
    cv=3,
    param_grid=params,
    verbose=3,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

dt_best = tree.DecisionTreeClassifier(
    random_state=42,
    max_depth=5,
    max_leaf_nodes=9,
    min_samples_split=2
)

dt_best.fit(X_train, y_train)

y_train_pred_dtbest = dt_best.predict(X_train)
y_test_pred_dtbest = dt_best.predict(X_test)

print(metrics.roc_auc_score(y_test, y_test_pred_dtbest))

Fitting 3 folds for each of 96 candidates, totalling 288 fits
{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}
0.7033229072349596


In [17]:
rf = ensemble.RandomForestClassifier(
    n_estimators=100,
    random_state=31
)

rf.fit(X_train, y_train)

y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

print(metrics.roc_auc_score(y_test_pred_rf, y_test))

0.8215242568663885


In [18]:
rf = ensemble.RandomForestClassifier(
    n_estimators=100,
    random_state=31
)

params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}

grid_search = model_selection.GridSearchCV(
    estimator=rf,
    cv=3,
    param_grid=params,
    verbose=3,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}


In [19]:
rf = ensemble.RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    max_features=7,
    min_samples_leaf=3,
    random_state=31
)

rf.fit(X_train, y_train)

y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

print(metrics.roc_auc_score(y_test_pred_rf, y_test))

0.8221378962697437


In [20]:
feature_names = [x for x in droped_data if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': rf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000


БУСТИНГ

In [21]:
air_data = pd.read_csv('data/AirPass.csv')
air_data.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [22]:
air_data = air_data.drop(['Unnamed: 0'], axis=1)
air_data.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [23]:
fill_list = {'Arrival Delay in Minutes':air_data['Arrival Delay in Minutes'].median()}
air_data.fillna(fill_list, inplace=True)

In [24]:
air_data['Arrival Delay in Minutes'].mean()

15.133392362180475

In [25]:
air_data['satisfaction'] = air_data['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
air_data['Customer Type'] = air_data['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
air_data['Type of Travel'] = air_data['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})

In [26]:
air_data_dummied = pd.get_dummies(air_data)
air_data_dummied.shape

(103904, 27)

In [27]:
X = air_data_dummied.drop(['satisfaction'], axis=1)
y = air_data_dummied['satisfaction']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=26)
X_test.shape

(20781, 26)

In [28]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 
X_test[0][0]

0.9408251379303

In [29]:
log_reg = linear_model.LogisticRegression()
log_reg.fit(X_train, y_train)

y_train_pred_logreg = log_reg.predict(X_train)
y_test_pred_logreg = log_reg.predict(X_test)

print(metrics.f1_score(y_test, y_test_pred_logreg))

0.8546883773161146


In [30]:
ada = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=26), random_state=26, learning_rate=0.01)
ada.fit(X_train, y_train)

y_train_pred_ada = ada.predict(X_train)
y_test_pred_ada = ada.predict(X_test)

print(metrics.f1_score(y_test, y_test_pred_ada))

0.9398901098901099


In [31]:
model_for_gs = ensemble.GradientBoostingClassifier()
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
gs = model_selection.GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  verbose=5)

gs.fit(X_train, y_train)

print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.886 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.883 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.885 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.901 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.899 total time=   0.4s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.897 total time=   0.5s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.916 total time=   1.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.911 total time=   0.9s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.912 total time=   0.8s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.932 total time=   1.5s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.932 total time=   1.8s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(random_state = 26)
xgb.fit(X_train, y_train)

y_train_pred_xgb = xgb.predict(X_train)
y_test_pred_xgb = xgb.predict(X_test)

print(metrics.f1_score(y_test, y_test_pred_xgb))

0.9579785161685312


In [ ]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_confusion_matrix

In [ ]:
cat = CatBoostClassifier()
cat.fit(X_train, y_train)

y_train_pred_cat = cat.predict(X_train)
y_test_pred_cat = cat.predict(X_test)

print(metrics.f1_score(y_test_pred_cat, y_test))

Learning rate set to 0.068023
0:	learn: 0.6018254	total: 61.8ms	remaining: 1m 1s
1:	learn: 0.5020788	total: 100ms	remaining: 50.1s
2:	learn: 0.4472585	total: 163ms	remaining: 54.2s
3:	learn: 0.4096053	total: 213ms	remaining: 53.2s
4:	learn: 0.3625882	total: 308ms	remaining: 1m 1s
5:	learn: 0.3353472	total: 369ms	remaining: 1m 1s
6:	learn: 0.3077169	total: 426ms	remaining: 1m
7:	learn: 0.2921065	total: 483ms	remaining: 59.8s
8:	learn: 0.2790138	total: 546ms	remaining: 1m
9:	learn: 0.2644628	total: 594ms	remaining: 58.8s
10:	learn: 0.2486162	total: 643ms	remaining: 57.8s
11:	learn: 0.2333939	total: 716ms	remaining: 58.9s
12:	learn: 0.2255875	total: 775ms	remaining: 58.9s
13:	learn: 0.2155887	total: 816ms	remaining: 57.4s
14:	learn: 0.2061545	total: 864ms	remaining: 56.7s
15:	learn: 0.2008754	total: 944ms	remaining: 58.1s
16:	learn: 0.1953161	total: 1.03s	remaining: 59.4s
17:	learn: 0.1889279	total: 1.09s	remaining: 59.4s
18:	learn: 0.1829712	total: 1.18s	remaining: 1m
19:	learn: 0.179001

In [ ]:
cm = get_confusion_matrix(cat, Pool(X_train, y_train))
cm

array([[46662.,   544.],
       [ 1285., 34632.]])

СТЕКИНГ

In [ ]:
frogs = pd.read_csv('Frogs_MFCCs.csv')
frogs.head()

In [ ]:
X = frogs.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y = frogs['Family']

X_train, X_test, y_train, y_test = model_selection.train_test_split(test_size=0.2, random_state=31)

In [ ]:
dt = tree.DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_test_pred_dt = dt.predict(X_test)

print(f'{round(metrics.f1_score(y_test, y_test_pred_dt), 3)}')

In [ ]:
# Модель стекинга
estimators = [
    ('rf', ensemble.RandomForestClassifier(n_estimators=10, random_state=31)),
    ('knn', sklearn.neighbors.KNeighborsClassifier(n_neighbors=11)),
    ('nb', sklearn.naive_bayes.GaussianNB())
]

metamodel = ensemble.StackingClassifier(
    estimators=estimators,
    final_estimator=linear_model.LogisticRegression()
)

metamodel.fit(X_train, y_train)
y_test_pred_meta = metamodel.predict(X_test)

print(f'{round(metrics.f1_score(y_test, y_test_pred_dt), 3)}')